In [26]:
import os
import pandas as pd
import numpy as np

In [4]:
pwd = os.getcwd()

filepath = pwd + '/Uncleaned_DS_jobs.csv'

raw_data = pd.read_csv(filepath) 

wip = raw_data.copy()

In [9]:
# replace \n with delimiter
wip['Company Name'] = wip['Company Name'].str.replace("\n", ",", regex=False)

# remove everything after delimiter
result = []
for item in wip['Company Name']:
    parts = item.split(',')
    result.append(','.join(parts[:1]))

wip['Company Name'] = result

In [6]:
# remove $
wip['SalaryEstUSD'] = wip['Salary Estimate'].str.replace('$', '', regex=False)

# add delimiter
# two formats present '''$145K-$225K (Glassdoor est.)''' and '''$145K-$225K(Employer est.)'''
wip['SalaryEstUSD'] = wip['SalaryEstUSD'].str.replace('(', ':', regex=False)

# remove everything after delimiter
result = []
for item in wip['SalaryEstUSD']:
    parts = item.split(':')
    result.append(':'.join(parts[:1]))

wip['SalaryEstUSD'] = result

In [7]:
# split into new columns
wip[['MinSalaryEstUSD', 'MaxSalaryEstUSD']] = wip['SalaryEstUSD'].str.split("-", expand=True)

# convert values
def convert_values(val):
    if 'K' in val:
        return float(val.replace('K', '')) * 1000
    else:
        return float(val)

wip['MinSalaryEstUSD'] = wip['MinSalaryEstUSD'].apply(convert_values)
wip['MaxSalaryEstUSD'] = wip['MaxSalaryEstUSD'].apply(convert_values)

# convert from float to int
wip['MinSalaryEstUSD'] = wip['MinSalaryEstUSD'].astype(int)
wip['MaxSalaryEstUSD'] = wip['MaxSalaryEstUSD'].astype(int)

In [56]:
# new column, only state job is located in
wip['Job State'] = wip['Location'].str.split(',').str[-1]

# new column, only state headquarters is located in
wip['HQ State'] = wip['Headquarters'].str.split(',').str[-1]

# new column, if job and HQ are located in the same state
wip['Same State'] = wip['Job State'] == wip['HQ State']

In [ ]:
# calculate company age
wip['Company Age'] = 2024 - wip['Founded']

# return sentinel values
wip['Company Age'] = wip['Company Age'].where(wip['Company Age'] != 2025, other='-1')

In [73]:
# remove listings without a rating (-1.0)
wip = wip[wip['Rating'] != -1]

In [79]:
clean = wip[['Job Title', 'Job Description', 'Rating','MinSalaryEstUSD', 'MaxSalaryEstUSD', 'Company Name', 'Location', 'Job State', 'Headquarters', 'Same State', 'Size', 'Founded','Company Age', 'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors']]

In [80]:
filepath_export = pwd + '/glassdoor_cleaned.csv'

clean.to_csv(filepath_export)